# LSTM and Dense

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler

In [ ]:
from keras.preprocessing import sequence
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten
import tensorflow as tf
from keras.utils import to_categorical

from keras import backend as K
from keras.models import Model
from keras.layers import Input, BatchNormalization, Dropout
import keras
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

## Import the training data

### Import training curves

In [ ]:
train_curves = pd.read_csv('../data/raw/training_set.csv')
print("train_curves columns: {}".format(train_curves.columns))
print("train_passbands shape: {}".format(train_curves.shape))
train_curves.head()

### Import training metadata

In [ ]:
train_metadata = pd.read_csv('../data/raw/training_set_metadata.csv')
print("train_metadata columns: {}".format(train_metadata.columns))
print("train_metadata shape: {}".format(train_metadata.shape))
train_metadata.head()

## Reorganize the data

### Merge curves and metadata into full train

In [ ]:
full_train = train_curves.reset_index(drop=True).merge(
    right=train_metadata,
    how='outer',
    on='object_id'
)
print("full_train columns: {}".format(full_train.columns))
print("full_train shape: {}".format(full_train.shape))
full_train.head()

### Create X_metadata

In [ ]:
gc.enable() # включение garbage collector

X_metadata = train_curves.copy() # считывание тренировочных данных (кривые)
X_metadata['flux_ratio_sq'] = np.power(X_metadata['flux'] / X_metadata['flux_err'], 2.0)
X_metadata['flux_by_flux_ratio_sq'] = X_metadata['flux'] * X_metadata['flux_ratio_sq']

aggs = {
    'mjd': ['min', 'max', 'size'],
    'passband': ['min', 'max', 'mean', 'median', 'std'],
    'flux': ['min', 'max', 'mean', 'median', 'std','skew'],
    'flux_err': ['min', 'max', 'mean', 'median', 'std','skew'],
    'detected': ['mean'],
    'flux_ratio_sq':['sum','skew'],
    'flux_by_flux_ratio_sq':['sum','skew'],
}

X_metadata = X_metadata.groupby('object_id').agg(aggs)
new_columns = [
    k + '_' + agg for k in aggs.keys() for agg in aggs[k]
]
X_metadata.columns = new_columns
X_metadata['mjd_diff'] = X_metadata['mjd_max'] - X_metadata['mjd_min']
X_metadata['flux_diff'] = X_metadata['flux_max'] - X_metadata['flux_min']
X_metadata['flux_dif2'] = (X_metadata['flux_max'] - X_metadata['flux_min']) / X_metadata['flux_mean']
X_metadata['flux_w_mean'] = X_metadata['flux_by_flux_ratio_sq_sum'] / X_metadata['flux_ratio_sq_sum']
X_metadata['flux_dif3'] = (X_metadata['flux_max'] - X_metadata['flux_min']) / X_metadata['flux_w_mean']

del X_metadata['mjd_max'], X_metadata['mjd_min']
gc.collect() # сбор мусора

print("X_metadata columns: {}".format(X_metadata.columns))
print("X_metadata shape: {}".format(X_metadata.shape))
X_metadata.head()

In [ ]:
X_metadata = X_metadata.reset_index().merge(
    right=train_metadata,
    how='outer',
    on='object_id'
)

if 'target' in X_metadata:
    y = X_metadata['target']
    del X_metadata['target']
classes = sorted(y.unique())

In [ ]:
if 'object_id' in X_metadata:
    oof_df = X_metadata[['object_id']]
    del X_metadata['object_id'], X_metadata['distmod'], X_metadata['hostgal_specz'] # удаление колонок
    del X_metadata['ra'], X_metadata['decl'], X_metadata['gal_l'], X_metadata['gal_b'], X_metadata['ddf'] # удаление колонок
    
print("X_metadata columns: {}".format(X_metadata.columns))
print("X_metadata shape: {}".format(X_metadata.shape))
X_metadata.head()

In [ ]:
ss = StandardScaler()
X_metadata_ss = ss.fit_transform(X_metadata)

In [ ]:
y = []
for name, group in full_train.groupby('object_id'):
    y.append(group['target'].iloc[0])
    
y = np.array(y)
y_classes = np.array(y)
y_classes

In [ ]:
y = pd.get_dummies(y)
print("y shape: {}".format(y.shape))

### Create X_curves

In [ ]:
X_temp = full_train[['object_id', 'passband', 'mjd', 'flux', 'flux_err', 'target']]
print("X_temp columns: {}".format(X_temp.columns))
print("X_temp shape: {}".format(X_temp.shape))
X_temp.head()

In [ ]:
%%time
X_curve = [[],[],[],[],[],[], [], [], [], [], [], []]
for name, group in X_temp.groupby(['object_id', 'passband']):
    X_curve[name[1]].append(group['flux'])
    X_curve[name[1] + 6].append(group['flux_err'])

In [ ]:
maxlen = 58

In [ ]:
%%time
X_curve = np.array([sequence.pad_sequences(x, maxlen=maxlen, dtype='float32', padding='post') for x in X_curve]).transpose(1, 2, 0)

In [ ]:
print("X_curve shape: {}".format(X_curve.shape))

## Creating the model

### Loss function for training

In [ ]:
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/69795
def mywloss(y_true,y_pred):  
    yc=tf.clip_by_value(y_pred,1e-15,1-1e-15)
    loss=-(tf.reduce_mean(tf.reduce_mean(y_true*tf.log(yc),axis=0)/wtable))
    return loss

### Loss function for poster evaluation

In [ ]:
def multi_weighted_logloss(y_ohe, y_p):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1-1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set 
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos    
    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss

### Class weight table

In [ ]:
# Костыль
wtable = np.array([0.01924057, 0.06307339, 0.117737  , 0.15201325, 0.02331804,
       0.00382263, 0.06167176, 0.01299694, 0.125     , 0.02650357,
       0.04714577, 0.29472477, 0.03045362, 0.02229867])

## Training the model

### Constants

In [ ]:
n_batch = 200
n_epoch = 50
n_features = 12
n_classes = y.shape[1]

In [ ]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1) # деление данных на фолды для кросс-валидации

In [ ]:
unique_y = np.unique(y_classes)
class_map = dict()
for i,val in enumerate(unique_y):
    class_map[val] = i

print(unique_y)
print()
print(class_map)
        
y_map = np.zeros((y_classes.shape[0],))
y_map = np.array([class_map[val] for val in y_classes])
y_categorical = to_categorical(y_map)

### LSTM with Dense on top

In [ ]:
K.clear_session()
def build_model(dropout_rate=0.25):
    
    curve_input = Input(shape=(None, n_features), name='curve_input')
    lstm_hidden = LSTM(256, return_sequences=True, dropout=0.1, name='lstm_1')(curve_input)
    lstm_out = LSTM(64, dropout=0.1, name='lstm_2')(lstm_hidden)
    
    metadata_input = Input(shape=(31,), name='metadata_input')
    x = keras.layers.concatenate([lstm_out, metadata_input], axis=1)
    
    dense_1 = Dense(512, activation='relu')(x)
    dense_1 = BatchNormalization()(dense_1)
    dense_1 = Dropout(dropout_rate)(dense_1)
    
    dense_2 = Dense(256, activation='relu')(dense_1)
    dense_2 = BatchNormalization()(dense_2)
    dense_2 = Dropout(dropout_rate)(dense_2)
    
    dense_3 = Dense(128, activation='relu')(dense_2)
    dense_3 = BatchNormalization()(dense_3)
    dense_3 = Dropout(dropout_rate)(dense_3)
    
    dense_4 = Dense(128, activation='relu')(dense_3)
    dense_4 = BatchNormalization()(dense_4)
    dense_4 = Dropout(dropout_rate)(dense_4)
    
    output = Dense(n_classes, activation='softmax', name='output')(dense_4)

    model = Model(inputs=[curve_input, metadata_input], outputs=output)
    return model

### LSTM and Dense with Dense on top

In [ ]:
K.clear_session()
def build_model_equal(dropout_rate=0.25):
    
    curve_input = Input(shape=(None, n_features), name='curve_input')
    
    lstm_hidden_1 = LSTM(256, return_sequences=True, dropout=0.1, name='lstm_1')(curve_input)
    lstm_hidden_2 = LSTM(64, dropout=0.1, name='lstm_2')(lstm_hidden_1)
    
    lstm_out = Dense(32)(lstm_hidden_2)
    
    metadata_input = Input(shape=(31,), name='metadata_input')
    
    dense_1 = Dense(512, activation='relu')(metadata_input)
    dense_1 = BatchNormalization()(dense_1)
    dense_1 = Dropout(dropout_rate)(dense_1)
    
    dense_2 = Dense(256, activation='relu')(dense_1)
    dense_2 = BatchNormalization()(dense_2)
    dense_2 = Dropout(dropout_rate)(dense_2)
    
    dense_3 = Dense(128, activation='relu')(dense_2)
    dense_3 = BatchNormalization()(dense_3)
    dense_3 = Dropout(dropout_rate)(dense_3)
    
    dense_4 = Dense(128, activation='relu')(dense_3)
    dense_4 = BatchNormalization()(dense_4)
    dense_4 = Dropout(dropout_rate)(dense_4)
    
    dense_out = Dense(32)(dense_4)
    
    x = keras.layers.concatenate([lstm_out, dense_out], axis=1)
    
    output_hidden = Dense(64)(x)
    output_hidden = BatchNormalization()(output_hidden)
    output_hidden = Dropout(dropout_rate)(output_hidden)
    
    output = Dense(n_classes, activation='softmax', name='output')(output_hidden)

    model = Model(inputs=[curve_input, metadata_input], outputs=output)
    return model

### Plot functions

In [ ]:
def plot_loss_acc(history):
    plt.plot(history.history['loss'][1:])
    plt.plot(history.history['val_loss'][1:])
    plt.title('model loss')
    plt.ylabel('val_loss')
    plt.xlabel('epoch')
    plt.legend(['train','Validation'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['acc'][1:])
    plt.plot(history.history['val_acc'][1:])
    plt.title('model Accuracy')
    plt.ylabel('val_acc')
    plt.xlabel('epoch')
    plt.legend(['train','Validation'], loc='upper left')
    plt.show()

### Training process

In [ ]:
clfs = []
oof_preds = np.zeros((len(X_metadata_ss), n_classes))
checkPoint = ModelCheckpoint("./keras_lstm.model",monitor='val_loss',mode = 'min', save_best_only=True, verbose=0)
for fold_, (trn_, val_) in enumerate(folds.split(y_map, y_map)):
    x_train, x_metadata, y_train = X_curve[trn_], X_metadata_ss[trn_], y.iloc[trn_]
    x_valid, x_metadata_val, y_valid = X_curve[val_], X_metadata_ss[val_], y.iloc[val_]
    
    model = build_model_equal()
    model.compile(loss=mywloss, optimizer='adam', metrics=['accuracy'])
    history = model.fit([x_train, x_metadata], y_train,
                    validation_data=[[x_valid, x_metadata_val], y_valid], 
                    epochs=n_epoch,
                    batch_size=n_batch, shuffle=True, verbose=2, callbacks=[checkPoint])  
    
    plot_loss_acc(history)
    
    print('Loading Best Model')
    model.load_weights('./keras_lstm.model')
    # # Get predicted probabilities for each class
    oof_preds[val_, :] = model.predict([x_valid, x_metadata_val],batch_size=n_batch)
    print(multi_weighted_logloss(y_valid, model.predict([x_valid, x_metadata_val], batch_size=n_batch)))
    clfs.append(model)


### Plotting the results

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_categorical,oof_preds))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_map, np.argmax(oof_preds,axis=-1))
np.set_printoptions(precision=2)

In [ ]:
sample_sub = pd.read_csv('../data/raw/sample_submission.csv')
class_names = list(sample_sub.columns[1:-1])
del sample_sub;gc.collect()


In [ ]:
# Plot non-normalized confusion matrix
plt.figure(figsize=(12,12))
foo = plot_confusion_matrix(cnf_matrix, classes=class_names,normalize=True,
                      title='Confusion matrix')

## Predicting on test data

In [ ]:
meta_test = pd.read_csv('../data/raw/test_set_metadata.csv') # считывание тестовых данных (кривые)

import time

temp_shit = 1

start = time.time()
chunks = 15000000
X_prev = np.array([[],[],[],[],[],[],[],[],[],[],[],[]])
for i_c, df in enumerate(pd.read_csv('../data/raw/test_set.csv', chunksize=chunks, iterator=True)):
    
    print(df.shape)
    print("Creating meta")
    df['flux_ratio_sq'] = np.power(df['flux'] / df['flux_err'], 2.0)
    df['flux_by_flux_ratio_sq'] = df['flux'] * df['flux_ratio_sq']
    # Group by object id
    agg_test = df.groupby('object_id').agg(aggs)
    agg_test.columns = new_columns
    agg_test['mjd_diff'] = agg_test['mjd_max'] - agg_test['mjd_min']
    agg_test['flux_diff'] = agg_test['flux_max'] - agg_test['flux_min']
    agg_test['flux_dif2'] = (agg_test['flux_max'] - agg_test['flux_min']) / agg_test['flux_mean']
    agg_test['flux_w_mean'] = agg_test['flux_by_flux_ratio_sq_sum'] / agg_test['flux_ratio_sq_sum']
    agg_test['flux_dif3'] = (agg_test['flux_max'] - agg_test['flux_min']) / agg_test['flux_w_mean']

    del agg_test['mjd_max'], agg_test['mjd_min']
#     del df
#     gc.collect()
    
    # Merge with meta data
    full_test = agg_test.reset_index().merge(
        right=meta_test,
        how='left',
        on='object_id'
    )
    full_test[full_train_new.columns] = full_test[full_train_new.columns].fillna(full_train_new.mean(axis=0))
    full_test_ss = ss.transform(full_test[full_train_new.columns])
    
    print("Creating curve")
    X_curve = df[['object_id', 'passband', 'mjd', 'flux', 'flux_err']]
    
#     print(type(X_prev))
    if isinstance(X_prev, (np.ndarray, np.generic)):
#         print("YES I AM A LIST AND ", type(X_prev))
        X_prev = X_prev.tolist()
    if isinstance(X_prev[0], (np.ndarray, np.generic)):
        for i in range(len(X_prev)):
            if isinstance(X_prev[i], (np.ndarray, np.generic)):
                X_prev[i] = X_prev[i].tolist()
    X_new = X_prev
#     print(X_prev[0])
    X_prev = [[],[],[],[],[],[],[],[],[],[],[],[]]
    for name, group in X_curve.groupby(['object_id', 'passband']):
#         print(name, ": ",group.shape)
        if len(group['flux']) != 0: 
            X_new[name[1]].append(group['flux'])
        else:
            X_new[name[1]].append([0])
#             print("asdfasdfasdf")
        if len(group['flux']) != 0: 
            X_new[name[1] + 6].append(group['flux_err'])
        else:
            X_new[name[1] + 6].append([0])
    
    print("Padding")
    
    for i, x in enumerate(X_new):
#         print("old: {}".format(len(x[0])))
        temp = sequence.pad_sequences(x, maxlen=maxlen, dtype='float32', padding='post')
#         print("new: {}".format(len(temp[0])))
        X_new[i] = temp
    
    for _ in range(20):
        for i in range(6):
            if len(X_new[i]) != min([len(x) for x in X_new]):
    #             print("STORING VALUES...")
                X_prev[i].append(X_new[i][-1])
                X_new[i] = np.delete(X_new[i], (-1), axis=0)

            if len(X_new[i+6]) != min([len(x) for x in X_new]):
    #             print("STORING VALUES...")
                X_prev[i+6].append(X_new[i+6][-1])
                X_new[i+6] = np.delete(X_new[i+6], (-1), axis=0)
        
    X_copy = np.array(X_new)
    print(X_copy.shape)
    for x in X_copy:
        print("shape: ", x.shape)
        
    X_copy = X_copy.transpose(1, 2, 0)
    #X_copy = np.array([sequence.pad_sequences(x, maxlen=maxlen, dtype='float32', padding='post') for x in X_new]).transpose(1, 2, 0)
    
    # Make predictions
    print("Making predictions")
    preds = None
    for clf in clfs:
        print("classifier...")
        if preds is None:
            preds = clf.predict([X_copy, full_test_ss]) / folds.n_splits
        else:
            preds += clf.predict([X_copy, full_test_ss]) / folds.n_splits
    
   # Compute preds_99 as the proba of class not being any of the others
    # preds_99 = 0.1 gives 1.769
    preds_99 = np.ones(preds.shape[0])
    for i in range(preds.shape[1]):
        preds_99 *= (1 - preds[:, i])
    
    # Store predictions
    preds_df = pd.DataFrame(preds, columns=class_names)
    preds_df['object_id'] = full_test['object_id']
    preds_df['class_99'] = 0.14 * preds_99 / np.mean(preds_99) 
    
    print("Writing to CSV")
    if i_c == 0:
        preds_df.to_csv('../data/submissions/predictions_lstm.csv',  header=True, mode='a', index=False)
    else: 
        preds_df.to_csv('../data/submissions/predictions_lstm.csv',  header=False, mode='a', index=False)
        
    del agg_test, full_test, preds_df, preds
#     print('done')
    if (i_c + 1) % 10 == 0:
        print('%15d done in %5.1f' % (chunks * (i_c + 1), (time.time() - start) / 60))